In [11]:
import json

def deduplicate_json(input_file, output_file):
    # 读取JSON文件
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 使用字典进行去重
    unique_dict = {}
    for item in data:
        key = (item['code'], item['name'])
        if key not in unique_dict:
            unique_dict[key] = item

    # 将去重后的数据转换回列表
    deduplicated_data = list(unique_dict.values())

    # 写入新的JSON文件
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(deduplicated_data, f, ensure_ascii=False, indent=2)

    print(f"原始数据数量: {len(data)}")
    print(f"去重后数据数量: {len(deduplicated_data)}")

# 使用函数
input_file = 'hushen300.json'
output_file = 'hushen300_deduplicated.json'
deduplicate_json(input_file, output_file)

原始数据数量: 300
去重后数据数量: 282


In [12]:
import akshare as ak
import pandas as pd

def get_hushen300_stocks():
    """
    获取沪深300成分股列表，返回包含股票代码和名称的DataFrame，
    其中股票代码列名为 'code'，股票名称列名为 'name'。
    对股票代码进行去重，保留首次出现的记录。
    """
    df = ak.index_stock_cons(symbol="000300")
    stocks = pd.DataFrame({
        'code': df['品种代码'],
        'name': df['品种名称']
    })
    # 根据code去重，保留第一次出现的记录
    stocks = stocks.drop_duplicates(subset=['code'], keep='first').reset_index(drop=True)
    return stocks

# 使用示例
hushen300_stocks = get_hushen300_stocks()
print(hushen300_stocks)
as_unique_codes = hushen300_stocks['code'].tolist()
print(type(as_unique_codes[0]))

       code  name
0    302132  中航成飞
1    001391   国货航
2    688047  龙芯中科
3    002600  领益智造
4    601077  渝农商行
..      ...   ...
277  600030  中信证券
278  600031  三一重工
279  600036  招商银行
280  600050  中国联通
281  000002   万科A

[282 rows x 2 columns]
<class 'str'>


# Diff两个json的代码

In [13]:
def compare_json(json1_path, json2_path):
    with open(json1_path, 'r', encoding='utf-8') as f1, open(json2_path, 'r', encoding='utf-8') as f2:
        data1 = json.load(f1)
        data2 = json.load(f2)

    code_data1 = {item["code"]: item for item in data1["signals"]}
    code_data2 = {item["code"]: item for item in data2["signals"]}

    all_codes = set(code_data1.keys()).union(set(code_data2.keys()))
    diffs = []
    for code in all_codes:
        item1 = code_data1.get(code, {})
        item2 = code_data2.get(code, {})
        keys1 = set(item1.keys())
        keys2 = set(item2.keys())
        all_keys = keys1.union(keys2)
        for key in all_keys:
            if key != "code" and item1.get(key) != item2.get(key):
                diffs.append({
                    "code": code,
                    "key": key,
                    "value_in_json1": item1.get(key),
                    "value_in_json2": item2.get(key)
                })
    return diffs

# 假设json1.json和json2.json是你要比较的两个文件路径
json1_path = '../log/signal-linux-qstock.json'
json2_path = '../log/signal-linux-akshare.json'
differences = compare_json(json1_path, json2_path)
for diff in differences:
    print(diff)

{'code': '300408', 'key': 'close', 'value_in_json1': 32.01, 'value_in_json2': None}
{'code': '300408', 'key': 'name', 'value_in_json1': '三环集团', 'value_in_json2': None}
{'code': '300408', 'key': 'date', 'value_in_json1': '2025-07-16', 'value_in_json2': None}
{'code': '300408', 'key': 'signals', 'value_in_json1': {'顶钝化': False, '底钝化': False, '顶结构': False, '底结构': False, '顶背离': False, '底背离': False, '主升': False, '顶成立': False, '底成立': False}, 'value_in_json2': None}
{'code': '000425', 'key': 'name', 'value_in_json1': '徐工机械', 'value_in_json2': '徐工科技'}
{'code': '688256', 'key': 'close', 'value_in_json1': 579.01, 'value_in_json2': None}
{'code': '688256', 'key': 'name', 'value_in_json1': '寒武纪', 'value_in_json2': None}
{'code': '688256', 'key': 'date', 'value_in_json1': '2025-07-16', 'value_in_json2': None}
{'code': '688256', 'key': 'signals', 'value_in_json1': {'顶钝化': False, '底钝化': False, '顶结构': False, '底结构': False, '顶背离': False, '底背离': False, '主升': False, '顶成立': False, '底成立': False}, 'value_in_

# QS接口的沪深300

In [19]:
import json

# 读取并提取股票代码（最简版本）
with open('../log/signal-linux-qstock.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取所有code并去重排序
qs_unique_codes = sorted(list(set(item['code'] for item in data['signals'])))

# 检查 as_unique_codes 中每个元素是否都出现在 qs_unique_codes 中
all_in = all(code in qs_unique_codes for code in as_unique_codes)
print(f"as_unique_codes 中的所有元素是否都在 qs_unique_codes 中: {all_in}")

as_unique_codes 中的所有元素是否都在 qs_unique_codes 中: True


In [29]:
# 找出qs_unique_codes中不在as_unique_codes中的code
diff_codes = [code for code in qs_unique_codes if code not in as_unique_codes]

# 根据data找出这些code对应的股票名字
diff_stocks = []
for item in data['signals']:
    if item['code'] in diff_codes:
        diff_stocks.append({'code': item['code'], 'name': item.get('name', '')})

print(len(diff_stocks))

18


In [26]:
print('601872' in as_unique_codes)

True


In [ ]:
import pandas as pd

# 创建示例DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': ['a', 'b', 'c', 'd', 'e'],
    'C': [10.1, 20.2, 30.3, 40.4, 50.5]
})

print("原始DataFrame:")
print(df)
print()

# 访问第2行第1列的元素（索引从0开始）
element = df.iloc[1, 0]  # 第2行，第1列
print(f"第2行第1列的元素: {element}")  # 输出: 2

# 访问第3行第2列的元素
element = df.iloc[2, 1]  # 第3行，第2列
print(f"第3行第2列的元素: {element}")  # 输出: c

In [ ]:
# 假设我们有以下股票数据
stock_data = pd.DataFrame({
    '日期': ['2025-07-15', '2025-07-16', '2025-07-17', '2025-07-18', '2025-07-19'],
    '开盘': [10.0, 10.2, 10.1, 10.3, 10.5],
    '最高': [10.5, 10.8, 10.6, 10.9, 11.0],
    '最低': [9.8, 10.0, 9.9, 10.1, 10.3],
    '收盘': [10.2, 10.6, 10.4, 10.7, 10.9],
    '成交量': [1000, 1200, 1100, 1300, 1400]
})

print("股票数据:")
print(stock_data)
print()

# 获取最新一天的数据（最后一行）
latest_data = stock_data.iloc[-1]
print("最新一天数据:")
print(latest_data)
print()

# 获取最近3天的收盘价
recent_closes = stock_data.iloc[-3:, 4]  # 最后3行，第5列（收盘价）
print("最近3天收盘价:")
print(recent_closes)
print()

# 获取第2-4天的所有数据
middle_data = stock_data.iloc[1:4, :]
print("第2-4天数据:")
print(middle_data)

In [18]:
def calculate_increase(now, old):
    if old == 0:
        return None
    increase = (now - old) / old * 100
    return f"{increase:.2f}%"

print(calculate_increase(5.45, 5.38))

1.30%
